In [ ]:
from jqdata import *
import pandas as pd
import re
# ... existing code ...

df = get_concepts()
fashion_concepts = []
for code, row in df.iterrows():
    # 假设 start_date 是字符串格式
    #if str(row['start_date']).startswith('2025'):
    if re.match(r'^(2024|2025)', str(row['start_date'])):
        continue
    # 过滤 name 包含"预增"的
    if re.search(r'预增', str(row['name'])):
        continue
    fashion_concepts.append((code, row['name']))
fashion_concepts = [(code, name) for code, name in fashion_concepts if '报预增' not in name]

#根据指定的concept获取对应的龙头股票
def get_head_stocks_on_concept(code, date):
    concept_stocks=get_concept_stocks(concept_code=code,date=date)
    df = get_fundamentals(query(
        valuation.code, valuation.market_cap, valuation.pe_ratio, income.total_operating_revenue).filter(
        valuation.code.in_(concept_stocks),
        valuation.market_cap > 1000,
        valuation.pe_ratio > 10,
        income.total_operating_revenue > 2e9).order_by(valuation.market_cap.desc()), date=date)
    #返回df的top3
    return df.head(3)

date = '2025-05-08'
all_heads = []
for code, name in fashion_concepts:
    df = get_head_stocks_on_concept(code, date)
    if df is not None and not df.empty:
        df['concept_code'] = code
        df['concept_name'] = name
        all_heads.append(df)

if all_heads:
    result_df = pd.concat(all_heads, ignore_index=True)
    print(result_df)
    # 写入文件，每行格式为 code,concept_code
    lines = [f"{row['code']},{row['concept_code']}" for _, row in result_df.iterrows()]
    write_file('dragon_equity_list.txt', '\n'.join(lines))
else:
    print("没有筛选出龙头股")

